# DBSCAN

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

In [3]:
#The dataset is quite large, and I am getting memory errors on this particular algorithm.
#I will split it randomnly.
df_features = df_features.sample(frac=0.05, random_state=763)

#We need to reset the index so the the algorithms below can loop through the dataframe
df_features = df_features.reset_index()

### Data Prep with t-SNE dimensionality reduction

Elaboration of my dimensionality reduction configuration decision process is given in the Final Dimension Reduction Configuration notebook.

In [4]:
#Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

In [5]:
# Loading the data
X = df_features
y = df_features.is_canceled

# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

X_norm = normalize(X_std)

In [6]:
from sklearn.manifold import TSNE
import time

In [7]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=100, n_iter=300)
tsne_results = tsne.fit_transform(X_std)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 1978 samples in 0.016s...
[t-SNE] Computed neighbors for 1978 samples in 0.298s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1978
[t-SNE] Computed conditional probabilities for sample 1978 / 1978
[t-SNE] Mean sigma: 1.125884
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.465439
[t-SNE] KL divergence after 300 iterations: 0.497333
t-SNE done! Time elapsed: 8.987588882446289 seconds


In the figure below, red shows the bookings that were not canceled, and blue shows the bookings that were.

# The Clustering Algorithms

The data should, in theory, have 2 natural clusters. Canceled and Not Canceled. However, the DBSCAN algorithm mapped it very cleanly in 5 clusters, so I will be trying in between 2 and 6 clusters for all the algorithms

In [8]:
#Libraries
from sklearn import datasets
from sklearn import metrics

## DBSCAN

DBSCAN did best with 2 eps regardless of the min_samples

In [9]:
from sklearn.cluster import DBSCAN

### The Best Performer

In [10]:
# Defining the DBSCAN clustering
dbscan_cluster = DBSCAN(eps=3, min_samples=3)

# Fit model
D_clusters = dbscan_cluster.fit_predict(tsne_results)

In [11]:
plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(D_clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

In [12]:
pd.crosstab(y, D_clusters)

col_0,0,1,2,3,4,5
is_canceled,,,,,,
0,1180,282,4,0,0,0
1,0,0,74,133,215,90


### Finding the Best Performer

In [13]:
# DBSCAN takes in eps, min_samples, x, and y 
def dbscan_ari_silh(eps, min_samples, x, y):
  from sklearn.cluster import DBSCAN
  if eps <= min_samples:
    pred = DBSCAN(eps, min_samples).fit_predict(x)
    print("Adjusted Rand Index of the DBSCAN solution with eps {} and min_samples {}: {}"
      .format(eps, min_samples, metrics.adjusted_rand_score(y, pred)))
    print("Silhouette score of the DBSCAN with eps {} and min_samples {}: {}".format(eps, min_samples,
      metrics.silhouette_score(x, pred, metric='euclidean')))
    print('='*90)
  else:
    diff = eps-min_samples
    min_samples = min_samples+diff
    pred = DBSCAN(eps, min_samples).fit_predict(x)
    print("Adjusted Rand Index of the DBSCAN solution with eps {} and min_samples {}: {}"
      .format(eps, min_samples, metrics.adjusted_rand_score(y, pred)))
    print("Silhouette score of the DBSCAN with eps {} and min_samples {}: {}".format(eps, min_samples,
      metrics.silhouette_score(x, pred, metric='euclidean')))
    print('='*90)

In [14]:
# check agglomerative clustering 
num_eps = np.arange(1,4)
num_min_samples = np.arange(3,11)

for i in num_eps:
  for j in num_min_samples:
    dbscan_ari_silh(i,j,tsne_results, y)
    print('\n')

Adjusted Rand Index of the DBSCAN solution with eps 1 and min_samples 3: 0.10904279046178719
Silhouette score of the DBSCAN with eps 1 and min_samples 3: 0.3102629780769348


Adjusted Rand Index of the DBSCAN solution with eps 1 and min_samples 4: 0.10905707406568682
Silhouette score of the DBSCAN with eps 1 and min_samples 4: 0.3308531641960144


Adjusted Rand Index of the DBSCAN solution with eps 1 and min_samples 5: 0.10844555613649152
Silhouette score of the DBSCAN with eps 1 and min_samples 5: 0.35031864047050476


Adjusted Rand Index of the DBSCAN solution with eps 1 and min_samples 6: 0.10713341274278416
Silhouette score of the DBSCAN with eps 1 and min_samples 6: 0.33490628004074097


Adjusted Rand Index of the DBSCAN solution with eps 1 and min_samples 7: 0.10688683483894215
Silhouette score of the DBSCAN with eps 1 and min_samples 7: 0.33235964179039


Adjusted Rand Index of the DBSCAN solution with eps 1 and min_samples 8: 0.10727045757190659
Silhouette score of the DBSCAN w